# Processing Data - SST 2

> https://huggingface.co/datasets/sst2

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

## Load the Glue SST 2 dataset

In [4]:
dataset = load_dataset('glue', 'sst2')

Generating test split: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 22208.99 examples/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [8]:
dataset['train'][42]

{'sentence': "as they come , already having been recycled more times than i 'd care to count ",
 'label': 0,
 'idx': 42}

In [11]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [12]:
def tokenize(data):
    return tokenizer(data['sentence'], truncation=True)

In [15]:
tokenized_datasets = dataset.map(tokenize, batched=True); tokenized_datasets

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 7699.69 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [17]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [41]:
samples = tokenized_datasets['train'][:8]

In [45]:
list(map(lambda x: len(x.split()), samples['sentence']))

[7, 7, 12, 8, 10, 10, 25, 2]

In [46]:
list(map(lambda x: len(x), samples['input_ids']))

[10, 11, 15, 10, 22, 13, 29, 6]

In [47]:
samples  = {k: v for k , v in samples.items() if k not in ['sentence', 'label', 'idx']}

In [48]:
batch = collator(samples)

In [49]:
for k, v in batch.items():
    print(k, v.shape)

input_ids torch.Size([8, 29])
token_type_ids torch.Size([8, 29])
attention_mask torch.Size([8, 29])


In [51]:
def generate_batch(tokenized_datasets, type='train', bs=32):
    ds = tokenized_datasets[type]
    for i in range(0, len(ds), bs):
        samples = ds[i: i + bs]
        samples  = {k: v for k , v in samples.items() if k not in ['sentence', 'label', 'idx']}
        yield collator(samples)

In [65]:
it = iter(generate_batch(tokenized_datasets))

In [74]:
batch = next(it)

In [75]:
batch.input_ids.shape

torch.Size([32, 49])

In [76]:
batch = next(it)

In [77]:
batch.input_ids.shape

torch.Size([32, 42])